# Part-of-Speech Tagging for Spanish

<div class="admonition note" name="html-admonition" style="background: lightblue; padding: 10px">
<p class="title">Note</p>
This section, "Working in Languages Beyond English," is co-authored with <a href="http://www.quinndombrowski.com/">Quinn Dombrowski</a>, the Academic Technology Specialist at Stanford University and a leading voice in multilingual digital humanities. I'm grateful to Quinn for helping expand this textbook to serve languages beyond English. 
</div>

In this lesson, we're going to learn about the textual analysis methods *part-of-speech tagging* and *keyword extraction* for Spanish-language texts. These methods will help us computationally parse sentences and better understand words in context.

---

## spaCy and Natural Language Processing (NLP)

To computationally identify parts of speech, we're going to use the natural language processing library spaCy. For a more extensive introduction to NLP and spaCy, see the previous lesson.

To parse sentences, spaCy relies on machine learning models that were trained on large amounts of labeled text data. If you've used the preprocessing or named entity recognition notebooks for this language, you can skip the steps for installing spaCy and downloading the language model.

## Install spaCy

To use spaCy, we first need to install the library.

In [ ]:
!pip install -U spacy

## Import Libraries

Then we're going to import `spacy` and `displacy`, a special spaCy module for visualization.

In [1]:
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
pd.options.display.max_rows = 400
pd.options.display.max_colwidth = 400

We're also going to import the `Counter` module for counting nouns, verbs, adjectives, etc., and the `pandas` library for organizing and displaying data (we're also changing the pandas default max row and column width display setting).

## Download Language Model

Next we need to download the Spanish-language model (`es_core_news_md`), which will be processing and making predictions about our texts. This is the model that was trained on the annotated ["AnCora" corpus](http://clic.ub.edu/corpus/). You can download the `es_core_news_md` model by running the cell below:

In [2]:
!python -m spacy download es_core_news_md

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---------------------------------------- 0.1/42.3 MB 2.4 MB/s eta 0:00:18
     ---------------------------------------- 0.4/42.3 MB 4.0 MB/s eta 0:00:11
      --------------------------------------- 0.7/42.3 MB 5.2 MB/s eta 0:00:08
     - -------------------------------------- 1.2/42.3 MB 6.5 MB/s eta 0:00:07
     - -------------------------------------- 1.6/42.3 MB 6.6 MB/s eta 0:00:07
     - -------------------------------------- 1.8/42.3 MB 6.2 MB/s eta 0:00:07
     - -------------------------------------- 1.9/42.3 MB 6.1 MB/s eta 0:00:07
     - -------------------------------------- 1.9/42.3 MB 6.1 MB/s eta 0:00:07
     -- ------------------------------------- 2.5/42.3 MB 6.0 MB/s eta 0:00:07
     -- ------------------------------------- 3.0/42.3 MB 6.4 MB/s eta 0:00:07
     --- ------------------------------------ 3.5/42.3 MB 6.8 MB/s eta 0:00:06
     --- ------------------------------------ 4.0/42.3 MB 7

*Note: spaCy offers [models for other languages](https://spacy.io/usage/models#languages) including Chinese, German, French, Spanish, Portuguese, Russian, Italian, Dutch, Greek, Norwegian, and Lithuanian*.  

*spaCy offers language and tokenization support for other language via external dependencies — such as [PyviKonlpy](https://github.com/konlpy/konlpy) for Korean.*

## Load Language Model

Once the model is downloaded, we need to load it with `spacy.load()` and assign it to the variable `nlp`.

In [3]:
nlp = spacy.load('es_core_news_md')

## Create a Processed spaCy Document

Whenever we use spaCy, our first step will be to create a processed spaCy `document` with the loaded NLP model `nlp()`. Most of the heavy NLP lifting is done in this line of code. After processing, the `document` object will contain tons of juicy language data — named entities, sentence boundaries, parts of speech — and the rest of our work will be devoted to accessing this information.

In [4]:
filepath = '../texts/es.txt'
text = open(filepath, encoding='utf-8').read()
document = nlp(text)

## spaCy Part-of-Speech Tagging
The tags that spaCy uses for part-of-speech are based on work done by [Universal Dependencies](https://universaldependencies.org/), an effort to create a set of part-of-speech tags that work across many different languages. Texts from various languages are annotated using this common set of tags, and contributed to a common repository that can be used to train models like spaCy.

The Universal Dependencies page has information about the annotated corpora available for each language; it's worth looking into the corpora that were annotated for your language.

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| ADJ   | adjective                 | big, old, green, incomprehensible, first      |
| ADP   | adposition                | in, to, during                                |
| ADV   | adverb                    | very, tomorrow, down, where, there            |
| AUX   | auxiliary                 | is, has (done), will (do), should (do)        |
| CONJ  | conjunction               | and, or, but                                  |
| CCONJ | coordinating conjunction  | and, or, but                                  |
| DET   | determiner                | a, an, the                                    |
| INTJ  | interjection              | psst, ouch, bravo, hello                      |
| NOUN  | noun                      | girl, cat, tree, air, beauty                  |
| NUM   | numeral                   | 1, 2017, one, seventy-seven, IV, MMXIV        |
| PART  | particle                  | ’s, not,                                      |
| PRON  | pronoun                   | I, you, he, she, myself, themselves, somebody |
| PROPN | proper noun               | Mary, John, London, NATO, HBO                 |
| PUNCT | punctuation               | ., (, ), ?                                    |
| SCONJ | subordinating conjunction | if, while, that                               |
| SYM   | symbol                    | $, %, §, ©, +, −, ×, ÷, =, :), 😝             |
| VERB  | verb                      | run, runs, running, eat, ate, eating          |
| X     | other                     | sfpksdpsxmsa                                  |
| SPACE | space                     |                                               |


Above is a POS chart taken from [spaCy's website](https://spacy.io/api/annotation#named-entities), which shows the different parts of speech that spaCy can identify as well as their corresponding labels. To quickly see spaCy's POS tagging in action, we can use the [spaCy module `displacy`](https://spacy.io/usage/visualizers#ent) on our sample `document` with the `style=` parameter set to "dep" (short for dependency parsing):

## Get Part-Of-Speech Tags

To get part of speech tags for every word in a document, we have to iterate through all the tokens in the document and pull out the `.lemma_` attribute for each token, which gives us the un-inflected version of the word. We'll also pull out the  `.pos_` attribute for each token. We can get even finer-grained dependency information with the attribute `.dep_`.


In [5]:
for token in document:
    print(token.lemma_, token.pos_, token.dep_)

introduccion NOUN ROOT
. PUNCT punct



 SPACE dep
economía NOUN ROOT
política ADJ amod
. PUNCT punct


 SPACE dep
el DET det
sombrío ADJ amod
Prudhon PROPN nsubj
, PUNCT punct
imbuído ADJ amod
, PUNCT punct
sin ADP advmod
duda NOUN fixed
, PUNCT punct
en ADP case
el DET det
idea NOUN obl
de ADP case
el DET det
Santos PROPN nmod

 SPACE dep
Padres PROPN flat
de ADP case
el DET det
Iglesia PROPN nmod
que PRON nsubj
predicar VERB acl
el DET det
desden ADP obj
por ADP case
el DET det
bien NOUN obl

 SPACE dep
terrenal ADJ amod
, PUNCT punct
decir VERB ROOT
que SCONJ mark
el DET det
pobreza NOUN nsubj
ser AUX cop
uno DET det
ley NOUN ccomp
de ADP case
nuestro DET det
naturaleza NOUN nmod
, PUNCT punct

 SPACE dep
ley NOUN appos
bajo ADP case
el DET det
cual PRON obl
haber AUX aux
ser AUX aux
constituer VERB acl
, PUNCT punct
de ADP case
donde PRON obl
él PRON expl:pass
deducir VERB acl
que SCONJ mark
el DET det

 SPACE dep
pauperismo NOUN nsubj
ser AUX cop
mal ADV advmod
que SCONJ mark
no 

## Practicing with the example text
When working with languages that have inflection, we typically use `token.lemma_` instead of `token.text` like you'll find in the English examples. This is important when we're counting, so that differently-inflected forms of a word (e.g. masculine vs. feminine or singular vs. plural) aren't counted as if they were different words.

In [6]:
filepath = "../texts/es.txt"
document = nlp(open(filepath, encoding="utf-8").read())

## Get Adjectives

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| ADJ   | adjective                 | big, old, green, incomprehensible, first      |

To extract and count the adjectives in the example text, we will follow the same model as above, except we'll add an `if` statement that will pull out words only if their POS label matches "ADJ."

<div class="admonition pythonreview" name="html-admonition" style="background: lightgreen; padding: 10px">
<p class="title">Python Review</p>

While we demonstrate how to extract parts of speech in the sections below, we're also going to reinforce some integral Python skills. Notice how we use `for` loops and `if` statements to `.append()` specific words to a list. Then we count the words in the list and make a pandas dataframe from the list.
</div>

Here we make a list of the adjectives identified in the example text:

In [7]:
adjs = []
for token in document:
    if token.pos_ == 'ADJ':
        adjs.append(token.lemma_)

In [8]:
adjs

['política',
 'sombrío',
 'imbuído',
 'terrenal',
 'desconsolado',
 'ingrato',
 'humano',
 'plausibl',
 'nuevo',
 'eficaz',
 'innúmera',
 'multiplicado',
 'social',
 'venerable',
 'entendido',
 'grave',
 '_',
 'político',
 '_',
 'moralizadoro',
 'servil',
 'colectivo',
 'precioso',
 'especulativo',
 'individual',
 'concluido',
 'abstracto',
 'ilustre',
 'elevado',
 'intelectual',
 'americano',
 'creador',
 'vivificador',
 'lujuriós',
 'invoco',
 'presente',
 'atrayente',
 'económico',
 'galanura',
 'natural',
 'social',
 'noble',
 'fuerte',
 'consolatorio',
 'arenoso',
 'privilegiado',
 'prestigioso',
 'suficiente',
 'soñadora',
 'adusta',
 '_',
 '_',
 'último',
 'marcado',
 'cajista',
 'extremo',
 'armado',
 'primero',
 'último',
 'último',
 '»--¿muere',
 'cariñoso',
 'pálido',
 'mojado',
 'primero',
 'aguardado',
 'dramático',
 'ademan',
 'forzado',
 'contento',
 'diario',
 'encorvado',
 'implacable',
 'terrible',
 'sano',
 'fuerte',
 'propuesto',
 'sentado',
 'pésimo',
 'querido',
 

Then we count the unique adjectives in this list with the `Counter()` module:

In [9]:
adjs_tally = Counter(adjs)

In [10]:
adjs_tally.most_common()

[('primero', 16),
 ('buen', 16),
 ('solo', 15),
 ('jóven', 12),
 ('último', 11),
 ('bello', 11),
 ('gran', 11),
 ('fuerte', 10),
 ('largo', 9),
 ('pobre', 9),
 ('único', 9),
 ('cuarto', 9),
 ('blanco', 8),
 ('mismo', 8),
 ('nuevo', 7),
 ('dulce', 7),
 ('viejo', 7),
 ('profundo', 7),
 ('nacional', 7),
 ('desconocido', 7),
 ('francés', 7),
 ('grande', 7),
 ('misterioso', 7),
 ('humano', 6),
 ('_', 6),
 ('querido', 6),
 ('necesario', 6),
 ('alto', 6),
 ('azul', 6),
 ('antiguo', 6),
 ('negro', 6),
 ('natural', 5),
 ('sentado', 5),
 ('mejor', 5),
 ('lindo', 5),
 ('rico', 5),
 ('verdadero', 5),
 ('amable', 5),
 ('mio', 5),
 ('amado', 5),
 ('social', 4),
 ('intelectual', 4),
 ('noble', 4),
 ('pálido', 4),
 ('contento', 4),
 ('adverso', 4),
 ('triste', 4),
 ('vestido', 4),
 ('central', 4),
 ('lejano', 4),
 ('distinguido', 4),
 ('benéfico', 4),
 ('á', 4),
 ('encerrado', 4),
 ('vecino', 4),
 ('dulcísimo', 4),
 ('cerrado', 4),
 ('grave', 3),
 ('precioso', 3),
 ('elevado', 3),
 ('presente', 3),
 (

Then we make a dataframe from this list:

In [11]:
df = pd.DataFrame(adjs_tally.most_common(), columns=['adj', 'count'])
df[:100]

,adj,count
0,primero,16
1,buen,16
2,solo,15
3,jóven,12
4,último,11
5,bello,11
6,gran,11
7,fuerte,10
8,largo,9
9,pobre,9


## Get Nouns

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| NOUN  | noun                      | girl, cat, tree, air, beauty                  |

To extract and count nouns, we can follow the same model as above, except we will change our `if` statement to check for POS labels that match "NOUN".

In [12]:
nouns = []
for token in document:
    if token.pos_ == 'NOUN':
        nouns.append(token.lemma_)

nouns_tally = Counter(nouns)

df = pd.DataFrame(nouns_tally.most_common(), columns=['noun', 'count'])
df[:100]

,noun,count
0,mano,48
1,señor,32
2,hora,28
3,casa,28
4,señora,26
5,trabajo,25
6,vida,24
7,padre,23
8,hombre,22
9,voz,22


## Get Verbs

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| VERB  | verb                      | run, runs, running, eat, ate, eating          |

To extract and count works of art, we can follow a similar-ish model to the examples above. This time, however, we're going to make our code even more economical and efficient (while still changing our `if` statement to match the POS label "VERB").

<div class="admonition pythonreview" name="html-admonition" style="background: lightgreen; padding: 10px">
<p class="title">Python Review</p>

We can use a [*list comprehension*](https://melaniewalsh.github.io/Intro-Cultural-Analytics/Python/More-Lists-Loops.html#List-Comprehensions) to get our list of verbs in a single line of code! Closely examine the first line of code below:
</div>

In [13]:
verbs = [token.lemma_ for token in document if token.pos_ == 'VERB']

verbs_tally = Counter(verbs)

df = pd.DataFrame(verbs_tally.most_common(), columns=['verb', 'count'])
df[:100]

,verb,count
0,hacer,55
1,decir,37
2,tener,33
3,dar,27
4,venir,24
5,ver,22
6,saber,20
7,querer,20
8,creer,19
9,hablar,19


# Keyword Extraction

## Get Sentences with Keyword

spaCy can also identify sentences in a document. To access sentences, we can iterate through `document.sents` and pull out the `.text` of each sentence.

We can use spaCy's sentence-parsing capabilities to extract sentences that contain particular keywords, such as in the function below. Note that the function assumes that the keyword provided will be exactly the same as it appears in the text (e.g. matching all inflection for case, number, gender, etc. As a Spanish example, if you use `bueno` as the keyboard, it won't match `buena` or `buenos`.)

With the function `find_sentences_with_keyword()`, we will iterate through `document.sents` and pull out any sentence that contains a particular "keyword." Then we will display these sentence with the keywords bolded.

In [14]:
import re
from IPython.display import Markdown, display

In [15]:
def find_sentences_with_keyword(keyword, document):
    
    #Iterate through all the sentences in the document and pull out the text of each sentence
    for sentence in document.sents:
        sentence = sentence.text
        
        #Check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
        if keyword.lower() in sentence.lower():
            
            #Use the regex library to replace linebreaks and to make the keyword bolded, again ignoring capitalization
            sentence = re.sub('\n', ' ', sentence)
            sentence = re.sub(f"{keyword}", f"**{keyword}**", sentence, flags=re.IGNORECASE)
            
            display(Markdown(sentence))

In [16]:
find_sentences_with_keyword(keyword="bueno", document=document)

Á «LA **bueno**S AIRES»                                        _LA AUTORA.

Enigma insoluble es para mí este jóven, tan franco, sin embargo, y tan **bueno**...  Y Emilio dejando la pluma en largo holgorio, quedóse engolfado en sus pensamientos y en el humo de su cigarro.     

Los exámenes; los premios; el paso á estudios superiores; el del vestido infantil al traje viril; la primera comunion.....  ¡Qué ceremonia, á la vez tan imponente y tierna!  Cumplido en él, el divino misterio, de rodillas ante el altar, el niño tiende la mano sobre el Sagrado Libro y jura ser virtuoso y **bueno**.  

Suscribíala el escribano D..., uno de los hombres más honorables de **bueno**s Aires.    

En el momento que Mauricio preparaba la realizacion de tan lisongero propósito, una carta de **bueno**s Aires, portadora de fatales nuevas, vino á destruir sus proyectos y sus esperanzas.    «Deber del hombre es ser fuerte y resignado, mi querido   Mauricio--decía el escribano D... en aquella carta.--Por tanto,   valor y resignacion.    

...

XXX   Antes de aquel término el Gerente de «La **bueno**s Aires» recibía una citacion del Banco Nacional con motivo del aviso que por su órden registraban «La Prensa» y «La Tribuna Nacional».  

Acudió el Gerente, y supo que allí se hallaba, depositado por el señor Cárlos Ridel, un paquete cerrado, que en Junio de 1888 debía ser entregado á la Compañía de Seguros «La **bueno**s Aires».  

Abierto el paquete, encontráronse con la póliza de seguro sobre la vida de Cárlos Ridel, por 20,000 $ m/n. en 20 años, y en la que constaba el abono de la primera anualidad de 792 $ m/n.; una carta á su esposa, y en defecto de ésta, á su hijo Mauricio; y una letra por 792 $ m/n. á la órden de Cárlos Ridel y endosada por éste á «La **bueno**s Aires», como la segunda cuota que debía pagar por su póliza.  

Entre estas, las Compañías de Seguros sónme especialmente   simpáticas, sobretodo, «La **bueno**s Aires», por su importancia y   valiosa organizacion.    

En el seno de una de esas asociaciones tutelares, «La **bueno**s Aires», la Providencia guardaba un tesoro que á su hora, hizo surgir para recompensar la abnegacion filial y dar la felicidad á los que, creyendo en ella, esperaban.  

## Get Keyword in Context

We can also find out about a keyword's more immediate context — its neighboring words to the left and right — and we can fine-tune our search with POS tagging.

To do so, we will first create a list of what's called *ngrams*. "Ngrams" are any sequence of *n* tokens in a text. They're an important concept in computational linguistics and NLP. (Have you ever played with [Google's *Ngram* Viewer](https://books.google.com/ngrams)?)

Below we're going to make a list of *bigrams*, that is, all the two-word combinations from the sample text. We're going to use these bigrams to find the neighboring words that appear alongside particular keywords.

In [17]:
#Make a list of tokens and POS labels from document if the token is a word 
tokens_and_labels = [(token.text, token.pos_) for token in document if token.is_alpha]

In [18]:
#Make a function to get all two-word combinations
def get_bigrams(word_list, number_consecutive_words=2):
    
    ngrams = []
    adj_length_of_word_list = len(word_list) - (number_consecutive_words - 1)
    
    #Loop through numbers from 0 to the (slightly adjusted) length of your word list
    for word_index in range(adj_length_of_word_list):
        
        #Index the list at each number, grabbing the word at that number index as well as N number of words after it
        ngram = word_list[word_index : word_index + number_consecutive_words]
        
        #Append this word combo to the master list "ngrams"
        ngrams.append(ngram)
        
    return ngrams

In [19]:
bigrams = get_bigrams(tokens_and_labels)

Let's take a peek at the bigrams:

In [20]:
bigrams[5:20]

[[('imbuído', 'ADJ'), ('sin', 'ADP')],
 [('sin', 'ADP'), ('duda', 'NOUN')],
 [('duda', 'NOUN'), ('en', 'ADP')],
 [('en', 'ADP'), ('las', 'DET')],
 [('las', 'DET'), ('ideas', 'NOUN')],
 [('ideas', 'NOUN'), ('de', 'ADP')],
 [('de', 'ADP'), ('los', 'DET')],
 [('los', 'DET'), ('Santos', 'PROPN')],
 [('Santos', 'PROPN'), ('Padres', 'PROPN')],
 [('Padres', 'PROPN'), ('de', 'ADP')],
 [('de', 'ADP'), ('la', 'DET')],
 [('la', 'DET'), ('Iglesia', 'PROPN')],
 [('Iglesia', 'PROPN'), ('que', 'PRON')],
 [('que', 'PRON'), ('predicaban', 'VERB')],
 [('predicaban', 'VERB'), ('el', 'DET')]]

Now that we have our list of bigrams, we're going to make a function `get_neighbor_words()`. This function will return the most frequent words that appear next to a particular keyword. The function can also be fine-tuned to return neighbor words that match a certain part of speech by changing the `pos_label` parameter.

In [21]:
def get_neighbor_words(keyword, bigrams, pos_label = None):
    
    neighbor_words = []
    keyword = keyword.lower()
    
    for bigram in bigrams:
        
        #Extract just the lowercased words (not the labels) for each bigram
        words = [word.lower() for word, label in bigram]        
        
        #Check to see if keyword is in the bigram
        if keyword in words:
            
            for word, label in bigram:
                
                #Now focus on the neighbor word, not the keyword
                if word.lower() != keyword:
                    #If the neighbor word matches the right pos_label, append it to the master list
                    if label == pos_label or pos_label == None:
                        neighbor_words.append(word.lower())
    
    return Counter(neighbor_words).most_common()

In [22]:
get_neighbor_words("bien", bigrams)

[('más', 2),
 ('que', 2),
 ('así', 1),
 ('fuerte', 1),
 ('y', 1),
 ('ya', 1),
 ('invisible', 1),
 ('pudiera', 1),
 ('distraido', 1),
 ('lo', 1),
 ('te', 1),
 ('cosa', 1),
 ('duerme', 1),
 ('charlábamos', 1),
 ('vá', 1),
 ('no', 1),
 ('si', 1),
 ('seguro', 1),
 ('mi', 1),
 ('amada', 1),
 ('tan', 1),
 ('esta', 1),
 ('palabra', 1),
 ('estaba', 1)]

In [23]:
get_neighbor_words("bien", bigrams, pos_label='VERB')

[('duerme', 1), ('charlábamos', 1), ('estaba', 1)]

## Your Turn!

Try out `find_sentences_with_keyword()` and `get_neighbor_words` with your own keywords of interest.

In [24]:
find_sentences_with_keyword(keyword="interesante", document=document)

¡Ah! ¿no era aquella ocasion para dar al diablo las prescripciones de la etiqueta, que le prohibían presentarse á sí propio á la **interesante** jóven, invocar el nombre del anciano Vice-Cónsul, y ofrecerla sus servicios?  

In [25]:
get_neighbor_words("interesante", bigrams, pos_label=None)

[('la', 1), ('jóven', 1)]